In [3]:
import tensorflow as tf
tf.keras.backend.clear_session()


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# ✅ Always clear session in Jupyter to avoid conflicts
tf.keras.backend.clear_session()

# -------------------
# Dataset Paths
# -------------------
train_dir = "../Data/FER/train"
val_dir   = "../Data/FER/test"

# -------------------
# Data Generators (RGB instead of grayscale)
# -------------------
base_datagen = ImageDataGenerator(rescale=1./255)

train_data = base_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode="rgb",   # ✅ use RGB
    class_mode="categorical",
    batch_size=32,      # fits into RTX 3050 (4GB)
    shuffle=True
)

val_data = base_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    color_mode="rgb",   # ✅ use RGB
    class_mode="categorical",
    batch_size=32,
    shuffle=False
)

# -------------------
# EfficientNetB0 Base (ImageNet Pretrained)
# -------------------
base_model = EfficientNetB0(
    weights="imagenet",        # ✅ pretrained ImageNet
    include_top=False,
    input_tensor=Input(shape=(48, 48, 3))   # ✅ 3 channels
)

# Freeze base model initially
base_model.trainable = False

# -------------------
# Custom Classification Head
# -------------------
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(train_data.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# -------------------
# Compile
# -------------------
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# -------------------
# Callbacks
# -------------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

# -------------------
# Phase 1: Train only classification head
# -------------------
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=callbacks
)

# -------------------
# Phase 2: Fine-tune EfficientNet
# -------------------
base_model.trainable = True   # unfreeze base
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # smaller LR for fine-tuning
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=callbacks
)

# -------------------
# Save Model
# -------------------
model.save("fer_efficientnetb0_rgb.h5")

# -------------------
# Plot Accuracy & Loss
# -------------------
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"] + history_ft.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"] + history_ft.history["val_accuracy"], label="Val Acc")
plt.legend(); plt.title("Accuracy")

# Loss
plt.subplot(1,2,2)
plt.plot(history.history["loss"] + history_ft.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"] + history_ft.history["val_loss"], label="Val Loss")
plt.legend(); plt.title("Loss")

plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '../Data/FER/train'

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input

model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_tensor=Input(shape=(224,224,3))   # 👈 default ImageNet input
)

print("EfficientNetB0 built successfully ✅")


ValueError: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (3, 3, 3, 32)

In [ ]:
    import tensorflow as tf
    from tensorflow.keras.applications import EfficientNetB0 # Or your desired EfficientNet variant

    # Create the model with 3 input channels (e.g., for RGB images)
    model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

ValueError: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (3, 3, 3, 32)

In [ ]:
import keras
import tensorflow as tf
print("keras version:", keras.__version__)
print("tf version:", tf.__version__)
print("keras location:", keras.__file__)
print("tf.keras location:", tf.keras.__file__)

keras version: 3.11.3
tf version: 2.20.0
keras location: /home/namithk/miniconda3/envs/studysense/lib/python3.10/site-packages/keras/__init__.py
tf.keras location: /home/namithk/miniconda3/envs/studysense/lib/python3.10/site-packages/keras/_tf_keras/keras/__init__.py
